# Redes neuronales para clasificación de imágenes
Considere el conjunto de imágenes de Emojis, las cuales son trazos de emojis de 5 tipos. Para estas imágenes, realice lo siguiente:

-Procesa cada una de las imágenes de tal manera que para cada imagen de color, se obtenga una imagen binaria de 32 x 32 donde el trazo principal del emoji esté centrado y ocupando el mayor espacio posible de la imagen. 

-Con las imágenes binarias, ajusta un modelo MLP y evalúa su rendimiento.

-Ajusta y evalúa una CNN para las imágenes binarias (25 puntos extras).

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [ ]:
path = 'Emojis'

emojis = []
labels = ['Angry', 'Happy', 'Poo', 'Sad', 'Surprised']
for label in labels:
    folder = os.path.join(path, label)
    for filename in os.listdir(folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                emojis.append((img, label))

print("Número total de imágenes cargadas:", len(emojis))

In [ ]:
# Muestra una imagen de cada categoría
plt.figure(figsize=(12, 8))
for i, label in enumerate(labels):
    for img, img_label in emojis:
        if img_label == label:
            plt.subplot(1, 5, i + 1)
            plt.imshow(img)
            plt.title(label)
            plt.axis('off')
            break

In [ ]:
# Preprocesamiento de las imágenes. 
def preprocess_image(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Imagen binaria (fondo blanco, trazos negros)
    binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        min_area = 30
        all_points = np.vstack([cnt for cnt in contours if cv2.contourArea(cnt) > min_area])
        x, y, w, h = cv2.boundingRect(all_points)
        roi = binary[y:y+h, x:x+w]  # Usar la imagen binaria
        roi_resized = cv2.resize(roi, (32, 32), interpolation=cv2.INTER_AREA)
        padded = np.pad(roi_resized, ((2, 2), (2, 2)), mode='constant', constant_values=0)
        return padded
    else:
        return cv2.resize(binary, (32, 32), interpolation=cv2.INTER_AREA)

preprocessed_emojis = [(preprocess_image(img), label) for img, label in emojis]
print("Número total de imágenes preprocesadas:", len(preprocessed_emojis))

In [ ]:
# Muestra una imagen de cada categoria preprocesada
plt.figure(figsize=(12, 8))
for i, label in enumerate(labels):
    for img, img_label in preprocessed_emojis:
        if img_label == label:
            plt.subplot(1, 5, i + 1)
            plt.imshow(img, cmap='gray')
            plt.title(label)
            plt.axis('off')
            break

In [ ]:
# Preparación de los datos para el modelo
X = np.array([img for img, label in preprocessed_emojis])
y_txt = np.array([labels.index(label)for img, label in preprocessed_emojis])
X = X.reshape(-1,36*36).astype('float32')/255.0
encoder = LabelEncoder()
y_num = encoder.fit_transform(y_txt)
y = to_categorical(y_num, num_classes=len(labels))
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

print("Forma de X_train:", X_train.shape)
print("Forma de y_train:", y_train.shape)
print("Forma de X_test:", X_test.shape)
print("Forma de y_test:", y_test.shape)



In [ ]:
# Define la funcion que crea el modelo MLP con keras
def create_mlp(input_dim, num_classes):
    model = Sequential()

    model.add(Dense(128, input_dim=input_dim, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [ ]:
# Entrena un modelo con los datos de entrenamiento
model = create_mlp(input_dim=36*36, num_classes=5)
model.summary()
model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)

In [ ]:
# Evalua el modelo con los datos de prueba
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)
print(classification_report(y_true_classes, y_pred_classes))

Entrenamiento utilizando una CNN

In [ ]:
#Preparación de Datos para la CNN

X_train_cnn = X_train.reshape(-1, 36, 36, 1)
X_test_cnn = X_test.reshape(-1, 36, 36, 1)



print("Forma de X_train para CNN:", X_train_cnn.shape)
print("Forma de X_test para CNN:", X_test_cnn.shape)
print("Forma de y_train para CNN:", y_train.shape) 
print("Forma de y_test para CNN:", y_test.shape) 

In [ ]:
# Genera la función que crea el modelo CNN
def create_cnn(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model    

In [ ]:
# Compila y entrena la CNN con los datos de entrenamiento
cnn_model = create_cnn(input_shape=(36, 36, 1), num_classes=5)
cnn_model.summary() 
cnn_model.fit(X_train_cnn, y_train, epochs=10, batch_size=32, validation_split=0.2, validation_data=(X_test_cnn, y_test))


In [ ]:
# Evalúa la CNN con los datos de prueba
y_pred_cnn = cnn_model.predict(X_test_cnn)
y_pred_classes_cnn = np.argmax(y_pred_cnn, axis=1)
print(classification_report(y_true_classes, y_pred_classes_cnn))